# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("../WeatherPy/output/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atuona,-9.80,-139.03,77.90,71,4,13.71,PF,1605197937
1,1,cabo san lucas,22.89,-109.91,75.00,60,20,6.93,MX,1605197672
2,2,busselton,-33.65,115.33,64.99,72,88,12.97,AU,1605197937
3,3,mercedes,-34.65,-59.43,84.00,57,0,1.99,AR,1605197937
4,4,cockburn town,21.46,-71.14,81.63,76,45,26.96,TC,1605197696


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configuring gmaps
gmaps.configure(api_key=g_key)

In [5]:
#adding heatmap
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity)
fig

SyntaxError: invalid syntax (<ipython-input-5-d98b239c76c2>, line 6)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#narrowing to fit conditions
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
98,98,esna,25.29,32.55,75.20,41,0,6.93,EG,1605197958
126,126,agadir,30.42,-9.60,71.60,64,0,9.17,MA,1605197964
135,135,veraval,20.90,70.37,78.48,48,0,2.19,IN,1605197965
154,154,phultala,22.70,89.52,72.05,65,0,4.72,BD,1605197971
215,215,maravilha,-26.76,-53.17,79.00,57,0,4.94,BR,1605197981
276,276,yulara,-25.24,130.99,73.40,17,0,5.82,AU,1605197996
328,328,mangrol,21.12,70.12,78.98,53,0,6.02,IN,1605198012
498,498,kruisfontein,-34.00,24.73,75.00,63,0,7.00,ZA,1605198051
561,561,noumea,-22.28,166.46,75.00,100,0,2.24,NC,1605197957


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
98,esna,EG,25.29,32.55,
126,agadir,MA,30.42,-9.60,
135,veraval,IN,20.90,70.37,
154,phultala,BD,22.70,89.52,
215,maravilha,BR,-26.76,-53.17,
276,yulara,AU,-25.24,130.99,
328,mangrol,IN,21.12,70.12,
498,kruisfontein,ZA,-34.00,24.73,
561,noumea,NC,-22.28,166.46,


In [19]:
# parameters for hotel
params = {
    "location": "0,0", 
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# iterating
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # request and url print, json conversion
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    # first hotel grab and store
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
98,esna,EG,25.29,32.55,
126,agadir,MA,30.42,-9.60,
135,veraval,IN,20.90,70.37,
154,phultala,BD,22.70,89.52,
215,maravilha,BR,-26.76,-53.17,
276,yulara,AU,-25.24,130.99,
328,mangrol,IN,21.12,70.12,
498,kruisfontein,ZA,-34.00,24.73,
561,noumea,NC,-22.28,166.46,


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))